In [20]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Funções

Função de conexão com o banco de dados

In [21]:
def connect_to_database(database):
    try:
        connection_string = f'mssql+pyodbc://localhost/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
        engine = create_engine(connection_string)
        return engine 
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        return None

Função para subir dados para o banco verificam se eles já existem pela chave

In [22]:
def send_to_db(engine, table, pk_column, dataset): 
    if not engine: 
        print('conexão fail')
        sys.exit()

    with engine.connect() as connection:
        for i, row in dataset.iterrows():
            query = text(f"select count(*) from {table} WHERE {pk_column} = :value")
            result = connection.execute(query, parameters=dict(value=row[pk_column]))
            count = result.scalar()

            if count > 0:
                print(f'Registro com {pk_column} = {row[pk_column]} já existe.')
            else: 
                row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
                print(f"Registro com {pk_column} = {row[pk_column]} inserido com sucesso.")
        connection.commit()
        connection.close()

Função de leitura de csv

In [23]:
def read_csv(file_path):
    dataset = pd.read_csv(file_path, sep='\t')

    return dataset

# Tabelas do banco

## Dimensões

TbAccount

In [24]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCabecalho.csv'
dataset = read_csv(file_path)

select_columns = [
    'id_account',
    'account_username',
    'account_biography',
    'profile_picture_url',
    'account_name',
    'followers_count', 
    'follows_count', 
    'media_count'
]
dataset= dataset[select_columns]


d_type = {
    'id_account': object
}
dataset = dataset.astype(d_type)

engine = connect_to_database('DW')
table = 'TbAccount'
pk_column = 'id_account'
send_to_db(engine, table, pk_column, dataset)


Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.


DTbDescricaoInsights

In [25]:
# Arquivo TbAccontDayInsights
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccontDayInsights.csv'
dataset = read_csv(file_path)
select_columns = [
    'name',
    'period',
    'title',
    'description'
]

## Herdar para carregar a tabela FTbAccountDayInsights
FTbAccountDayInsights = dataset


dataset_day_dimension = dataset[select_columns]

# Arquivos TbAccountLifeCidadeDosSeguidores, TbAccountLifeFaixaGenero, TbAccountLifeLocaPais, TbAccountLifePaisDosSeguidores, TbAccountLifeSeguidoresOnline
cidade_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeCidadeDosSeguidores.csv'
Faixa_genero = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeFaixaGenero.csv'
local_pais = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeLocaPais.csv'
pais_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifePaisDosSeguidores.csv'
seguidores_on = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeSeguidoresOnline.csv'

list_df = [cidade_seguidores, Faixa_genero, local_pais, pais_seguidores, seguidores_on]
dfs = [] 

for df in list_df: 
    data = pd.read_csv(df, sep='\t')
    dfs.append(data)

dataset = pd.concat(dfs)
select_columns = [
    'name',
    'period',
    'title',
    'description'
]

### Usar como herença para tabela FTbAccountLifetimeInsights
FTbAccountLifetimeInsights = dataset


dataset_lifetime_dimension = dataset[select_columns]


## TbStoriesInsights
stories = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStoriesInsights.csv'
dataset_1 = read_csv(stories)
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
stories_descricao_dimensao = dataset_1[select_columns]


#TbMidiasInsights
midias = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMidiasInsights.csv'
dataset_2 = read_csv(midias)
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
midias_descricao_dimensao = dataset_2[select_columns]


## Transformando em dados únicos
dataset_day_dimension.drop_duplicates(inplace=True)
dataset_lifetime_dimension.drop_duplicates(inplace=True)
stories_descricao_dimensao.drop_duplicates(subset=('name'), inplace=True)
midias_descricao_dimensao.drop_duplicates(subset=('name'),inplace=True)


## Unindo todas as dimensões 
DTbAccountInsights = pd.concat([dataset_day_dimension,dataset_lifetime_dimension, stories_descricao_dimensao, midias_descricao_dimensao])
DTbAccountInsights.sort_values('name', inplace=True)
DTbAccountInsights.drop_duplicates(subset=('name'),inplace=True)
DTbAccountInsights.reset_index(inplace=True)
DTbAccountInsights['id_insight'] = DTbAccountInsights.index + 1
DTbAccountInsights = DTbAccountInsights[['id_insight'] + [col for col in DTbAccountInsights.columns if col != 'id_insight']]
DTbAccountInsights.drop(columns='index', inplace=True)
DTbAccountInsights.rename(columns={'period': 'frequencia'}, inplace=True)

engine = connect_to_database('DW')
table = 'DTbDescricaoInsights'
pk_column = 'id_insight'

send_to_db(engine, table, pk_column, DTbAccountInsights)

Registro com id_insight = 1 já existe.
Registro com id_insight = 2 já existe.
Registro com id_insight = 3 já existe.
Registro com id_insight = 4 já existe.
Registro com id_insight = 5 já existe.
Registro com id_insight = 6 já existe.
Registro com id_insight = 7 já existe.
Registro com id_insight = 8 já existe.
Registro com id_insight = 9 já existe.
Registro com id_insight = 10 já existe.
Registro com id_insight = 11 já existe.
Registro com id_insight = 12 já existe.
Registro com id_insight = 13 já existe.
Registro com id_insight = 14 já existe.
Registro com id_insight = 15 já existe.
Registro com id_insight = 16 já existe.
Registro com id_insight = 17 já existe.
Registro com id_insight = 18 já existe.
Registro com id_insight = 19 já existe.
Registro com id_insight = 20 já existe.
Registro com id_insight = 21 já existe.


C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\2524290354.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_day_dimension.drop_duplicates(inplace=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\2524290354.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_lifetime_dimension.drop_duplicates(inplace=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\2524290354.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

DTbMidias

In [26]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMedias.csv'
dataset = pd.read_csv(file_path, sep='\t')

## Aproveitar carregamento para subir FTDMidias
FTbMidias = dataset

select_columns = [
    'id_midia',
    'id_account',
    'media_type',
    'media_url',
    'caption',
    'timestamp',
    'permalink',
    'media_product_type',
    'thumbnail_url',
    'shortcode'
]
DTbMidias = dataset[select_columns]

# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
# # Agora, vamos extrair apenas a data (sem a hora) das colunas de data
# DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date
type_columns = {
    'id_midia':  'object',
    'id_account': 'object',
}
DTbMidias = DTbMidias.astype(type_columns)

engine = connect_to_database('DW')
table = 'DTbMidias'
pk_column = 'id_midia'
send_to_db(engine, table, pk_column, DTbMidias)

Registro com id_midia = 18005273918369940 inserido com sucesso.
Registro com id_midia = 17846370906157882 inserido com sucesso.
Registro com id_midia = 18002752616172667 inserido com sucesso.


C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\4042909499.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\4042909499.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date


Registro com id_midia = 17971594208670851 inserido com sucesso.
Registro com id_midia = 18062694892504235 inserido com sucesso.
Registro com id_midia = 17870243565059503 inserido com sucesso.
Registro com id_midia = 17897388878957061 inserido com sucesso.
Registro com id_midia = 18214911715272967 inserido com sucesso.
Registro com id_midia = 17905338809918494 inserido com sucesso.
Registro com id_midia = 17912491130893668 inserido com sucesso.
Registro com id_midia = 18020750125974446 inserido com sucesso.
Registro com id_midia = 17869447275058315 inserido com sucesso.
Registro com id_midia = 18030343753699664 inserido com sucesso.
Registro com id_midia = 18012618164496765 inserido com sucesso.
Registro com id_midia = 18020150257990451 inserido com sucesso.
Registro com id_midia = 18024592819898084 inserido com sucesso.
Registro com id_midia = 17997324461240753 inserido com sucesso.
Registro com id_midia = 17982183917471535 inserido com sucesso.
Registro com id_midia = 1787317315502572

In [27]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStories.csv'
dataset = pd.read_csv(file_path, sep='\t')
stories = dataset

select_columns = [
    'id_midia',
    'id_account',
    'media_type',
    'media_url',
    'caption',
    'timestamp',
    'permalink',
    'media_product_type',
    'thumbnail_url',
    'shortcode'
]

if 'thumbnail_url' not in dataset.columns:
    dataset['thumbnail_url'] = pd.Series(dtype=object)

DTbMidias = dataset[select_columns]

# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])

# # Agora, vamos extrair apenas a data (sem a hora) das colunas de data
# DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date

type_columns = {
    'id_midia':  'object',
    'id_account': 'object',
}
DTbMidias = DTbMidias.astype(type_columns)


engine = connect_to_database('DW')
table = 'DTbMidias'
pk_column = 'id_midia'
send_to_db(engine, table, pk_column, DTbMidias)

Registro com id_midia = 18029303027098233 inserido com sucesso.


C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\3692403429.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\3692403429.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date


FTbMidias
Vou unir o dataframe da TbMidas e dos stories para subir na mesa tabela pois ambos se tratam de midas

In [28]:
select_columns = [
    'id_midia',
    'id_account',
    'comments_count',
    'like_count',
    'extract_date',
    'period',
    'year',
    'day'
]

df_1 = FTbMidias[select_columns]
df_2 = stories[select_columns]
FTbMidias__ = pd.concat([df_1, df_2], ignore_index=True)


FTbMidias__['id'] = FTbMidias__['id_midia'].astype(str) + \
    FTbMidias__['id_account'].astype(str) + FTbMidias__['year'].astype(str) \
          + FTbMidias__['period'].astype(str) + FTbMidias__['day'].astype(str)
FTbMidias__ = FTbMidias__[['id','id_midia', 'id_account', 'comments_count', 'like_count', 'extract_date', 'period', 'year', 'day']]

FTbMidias__['extract_date'] = pd.to_datetime(FTbMidias__['extract_date'])
FTbMidias__['extract_date'] = FTbMidias__['extract_date'].dt.date

type_columns = {
    'id_midia': 'object',
    'id_account': 'object',
    'comments_count': 'int',
    'like_count': 'int',
    'period': 'object',
    'year': 'object',
    'day': 'object'
}

FTbMidias__ = FTbMidias__.astype(type_columns)

engine = connect_to_database('DW')
table = 'FTbMidias'
pk_column = 'id'
send_to_db(engine, table, pk_column, FTbMidias__)

Registro com id = 18005273918369940178414254445161882024524 inserido com sucesso.
Registro com id = 17846370906157882178414254445161882024524 inserido com sucesso.
Registro com id = 18002752616172667178414254445161882024524 inserido com sucesso.
Registro com id = 17971594208670851178414254445161882024524 inserido com sucesso.
Registro com id = 18062694892504235178414254445161882024524 inserido com sucesso.
Registro com id = 17870243565059503178414254445161882024524 inserido com sucesso.
Registro com id = 17897388878957061178414254445161882024524 inserido com sucesso.
Registro com id = 18214911715272967178414254445161882024524 inserido com sucesso.
Registro com id = 17905338809918494178414254445161882024524 inserido com sucesso.
Registro com id = 17912491130893668178414254445161882024524 inserido com sucesso.
Registro com id = 18020750125974446178414254445161882024524 inserido com sucesso.
Registro com id = 17869447275058315178414254445161882024524 inserido com sucesso.
Registro com id 

FTbAccountDayInsights

In [29]:
# FTbAccountDayInsights.head()
# Unindo com a tb de dimensão para verificar
engine = connect_to_database('dw')
query = "select * from DTbDescricaoInsights"
DTbDescricaoInsights = pd.read_sql(query, engine)

FTbAccountDayInsights = pd.merge(FTbAccountDayInsights, DTbDescricaoInsights, on='name', how= 'inner', suffixes=('_left', '_right'))

select_columns = [
  'id_tb_midia',
  'id_insight',
  'id_account',
  'last_day',
  'last_end_time',
  'actual_day',
  'actual_end_time',
  'extract_date',
  'period_extraction',
  'year',
  'day'
 ]
FTbAccountDayInsights = FTbAccountDayInsights[select_columns]


columns_rename = {
    'id_tb_midia': 'id',
    'last_day': 'value_last_day',
    'actual_day': 'value_actual_day'
}
FTbAccountDayInsights.rename(columns = columns_rename, inplace=True)


# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
FTbAccountDayInsights['last_end_time'] = pd.to_datetime(FTbAccountDayInsights['last_end_time'])
FTbAccountDayInsights['actual_end_time'] = pd.to_datetime(FTbAccountDayInsights['actual_end_time'])
FTbAccountDayInsights['extract_date'] = pd.to_datetime(FTbAccountDayInsights['extract_date'])

# Agora, vamos extrair apenas a data (sem a hora) das colunas de data
FTbAccountDayInsights['last_end_time'] = FTbAccountDayInsights['last_end_time'].dt.date
FTbAccountDayInsights['actual_end_time'] = FTbAccountDayInsights['actual_end_time'].dt.date
FTbAccountDayInsights['extract_date'] = FTbAccountDayInsights['extract_date'].dt.date

type_columns = {
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object'                
}

FTbAccountDayInsights = FTbAccountDayInsights.astype(type_columns)


engine = connect_to_database('DW')
table = 'FTbAccountDayInsights'
pk_column = 'id'
send_to_db(engine, table, pk_column, FTbAccountDayInsights)

Registro com id = 1784142544451618820240524impressionsday inserido com sucesso.
Registro com id = 1784142544451618820240524reachday inserido com sucesso.
Registro com id = 1784142544451618820240524follower_countday inserido com sucesso.
Registro com id = 1784142544451618820240524email_contactsday inserido com sucesso.
Registro com id = 1784142544451618820240524phone_call_clicksday inserido com sucesso.
Registro com id = 1784142544451618820240524text_message_clicksday inserido com sucesso.
Registro com id = 1784142544451618820240524get_directions_clicksday inserido com sucesso.
Registro com id = 1784142544451618820240524website_clicksday inserido com sucesso.
Registro com id = 1784142544451618820240524profile_viewsday inserido com sucesso.
Registro com id = 1784141753040061620240524impressionsday inserido com sucesso.
Registro com id = 1784141753040061620240524reachday inserido com sucesso.
Registro com id = 1784141753040061620240524follower_countday inserido com sucesso.
Registro com i

FTbMidiasInsights

In [30]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStoriesInsights.csv'
dataset_1 = pd.read_csv(file_path, sep='\t')
TbStoriesInsights = dataset_1

file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMidiasInsights.csv'
dataset_2 = pd.read_csv(file_path, sep='\t')
TbMidiasInsights = dataset_2
# TbMidiasInsights.rename(columns={'id': 'id_midia'}, inplace=True)


engine = connect_to_database('dw')
query = "select * from DTbDescricaoInsights"
DTbDescricaoInsights = pd.read_sql(query, engine)


TbStoriesInsights = pd.merge(TbStoriesInsights, DTbDescricaoInsights, on='name', how= 'inner')
TbStoriesInsights.rename(columns={'id_insight_x': 'id_insight'}, inplace=True)
TbMidiasInsights = pd.merge(TbMidiasInsights, DTbDescricaoInsights, on='name', how= 'inner')

select_columns = [
    'id_midia',
    'id_insight',
    'extract_date',
    'period_extraction',
    'year',
    'day',
    'name',  # Renomear
    'value'
]

TbStoriesInsights = TbStoriesInsights[select_columns]
TbMidiasInsights = TbMidiasInsights[select_columns]

FTbMidiasInsights = pd.concat([TbStoriesInsights, TbMidiasInsights], ignore_index=True)
FTbMidiasInsights['id'] = FTbMidiasInsights['id_midia'].astype(str) + FTbMidiasInsights['id_insight'].astype(str) + \
                            FTbMidiasInsights['year'].astype(str) + FTbMidiasInsights['period_extraction'].astype(str) + \
                                FTbMidiasInsights['day'].astype(str)

FTbMidiasInsights = FTbMidiasInsights[['id','id_midia', 'id_insight', 'extract_date', 'period_extraction', 'year', 'day', 'name', 'value']]
FTbMidiasInsights.rename(columns={'name': 'definicao'}, inplace=True)

FTbMidiasInsights['extract_date'] = pd.to_datetime(FTbMidiasInsights['extract_date'])
FTbMidiasInsights['extract_date'] = FTbMidiasInsights['extract_date'].dt.date

type_columns = {
    'id_midia': 'object',
    'id_insight': 'object',
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object',
    'value': 'int'
}

FTbMidiasInsights = FTbMidiasInsights.astype(type_columns)

engine = connect_to_database('DW')
table = 'FTbMidiasInsights'
pk_column = 'id'
send_to_db(engine, table, pk_column, FTbMidiasInsights)

Registro com id = 1802930302709823372024523 inserido com sucesso.
Registro com id = 18029303027098233152024523 inserido com sucesso.
Registro com id = 18029303027098233182024523 inserido com sucesso.
Registro com id = 18029303027098233172024523 inserido com sucesso.
Registro com id = 18029303027098233102024523 inserido com sucesso.
Registro com id = 18029303027098233142024523 inserido com sucesso.
Registro com id = 18005273918369940142024524 inserido com sucesso.
Registro com id = 18005273918369940162024524 inserido com sucesso.
Registro com id = 17846370906157882142024524 inserido com sucesso.
Registro com id = 17846370906157882162024524 inserido com sucesso.
Registro com id = 18002752616172667142024524 inserido com sucesso.
Registro com id = 18002752616172667162024524 inserido com sucesso.
Registro com id = 17971594208670851142024524 inserido com sucesso.
Registro com id = 17971594208670851162024524 inserido com sucesso.
Registro com id = 18062694892504235142024524 inserido com suces

FTbAccountLifetimeInsights

In [31]:
engine = connect_to_database('dw')
query = "select * from DTbDescricaoInsights"
DTbDescricaoInsights = pd.read_sql(query, engine)

cidade_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeCidadeDosSeguidores.csv'
Faixa_genero = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeFaixaGenero.csv'
local_pais = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeLocaPais.csv'
pais_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifePaisDosSeguidores.csv'
seguidores_on = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeSeguidoresOnline.csv'

list_df = [cidade_seguidores, Faixa_genero, local_pais, pais_seguidores, seguidores_on]
dfs = [] 

for df in list_df: 
    data = pd.read_csv(df, sep='\t')
    dfs.append(data)

dataset = pd.concat(dfs)


FTbAccountLifetimeInsights = dataset

FTbAccountLifetimeInsights = pd.merge(FTbAccountLifetimeInsights, DTbDescricaoInsights, on='name', how= 'inner')

select_columns = [
    'id_tb_account',
    'id_account',
    'id_insight',
    'extract_date',
    'period_extraction',
    'year',
    'day',
    'name',
    'age_gender',
    'value'
]

FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[select_columns]

rename_columns = {
    'id_tb_account': 'id',
    'name': 'definicao',
    'age_gender': 'value_descricao'
}

FTbAccountLifetimeInsights.rename(columns=rename_columns, inplace=True)
FTbAccountLifetimeInsights['value_descricao'] = FTbAccountLifetimeInsights['value_descricao'].str.replace('value.', '')


def replace_non_numeric(value):
    try:
        float(value)  # Tenta converter o valor para float
        return value  # Se for numérico, mantenha o valor
    except ValueError:
        return 0  # Se não for numérico, substitua por 0

# Aplique a função à coluna 'value'
FTbAccountLifetimeInsights['value'] = FTbAccountLifetimeInsights['value'].apply(lambda x: replace_non_numeric(x))
FTbAccountLifetimeInsights['value'] = FTbAccountLifetimeInsights['value'].fillna(0)


FTbAccountLifetimeInsights['extract_date'] = pd.to_datetime(FTbAccountLifetimeInsights['extract_date'])

type_columns = {
    'id_account': 'object',
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object'
}

FTbAccountLifetimeInsights = FTbAccountLifetimeInsights.astype(type_columns)
FTbAccountLifetimeInsights.drop(columns='id', inplace=True)
FTbAccountLifetimeInsights['id'] = FTbAccountLifetimeInsights['id_account'].astype(str) + FTbAccountLifetimeInsights['id_insight'].astype(str) +  FTbAccountLifetimeInsights['extract_date'].astype(str) + FTbAccountLifetimeInsights['value_descricao'] 

colunas = ['id'] + [coluna for coluna in FTbAccountLifetimeInsights if coluna != 'id']
FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[colunas]


FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[FTbAccountLifetimeInsights['value'] != 0]
FTbAccountLifetimeInsights.drop_duplicates(inplace=True)



engine = connect_to_database('DW')
table_name = 'FTbAccountLifetimeInsights'
key_column = 'id'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in FTbAccountLifetimeInsights.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        try:
            if count > 0:
                print(f'Registro com {key_column} = {row[key_column]} já existe.')
            else: 
                row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
                print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
        except:
            pass
    connection.commit()
    connection.close()

Registro com id = 1784142544451618812024-05-24São Paulo, São Paulo (state) inserido com sucesso.
Registro com id = 1784141753040061612024-05-24São Paulo, São Paulo (state) inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Karabük, Karabük Province inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Rio de Janeiro, Rio de Janeiro (state) inserido com sucesso.
Registro com id = 1784141753040061612024-05-24Rio de Janeiro, Rio de Janeiro (state) inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Yuma, Arizona inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Edinburg, Texas inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Ulukısla, Niğde Province inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Abington, Pennsylvania inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Sioux Falls, South Dakota inserido com sucesso.
Registro com id = 1784142544451618812024-05-24Golpazar

OperationalError: (pyodbc.OperationalError) ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Provedor de Memória Compartilhada: Não há processo no outro extremo do pipe.\r\n (233) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Falha de vínculo de comunicação (233)')
[SQL: select count(*) from FTbAccountLifetimeInsights WHERE id = ?]
[parameters: ('1784142544451618812024-05-24Cape Town, Western Cape',)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

# DTbCarrocel

In [38]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCarrossel.csv'
dataset = pd.read_csv(file_path, sep='\t', dtype={'id_midia': str})
rename_columns = {
    'media_type.1': 'type'
}
dataset.rename(columns=rename_columns, inplace=True)
dataset.drop(columns=('Unnamed: 0'),inplace=True)
dataset['id'] = dataset['id'].astype(str) + dataset['id_midia'].astype(str)
dataset.dropna(subset=('id_midia'), inplace=True)

table = 'DTbcarrossel'
pk_column = 'id'
engine = connect_to_database('dw')
send_to_db(engine, table, pk_column, dataset)
# dataset.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table,

Registro com id = 1797182902765935718024592819898084 inserido com sucesso.
Registro com id = 1797182902765935717950801427740207 inserido com sucesso.
Registro com id = 1797182902765935718249982000212022 inserido com sucesso.
Registro com id = 1797182902765935717970826016660160 inserido com sucesso.
Registro com id = 1797182902765935717983677587408251 inserido com sucesso.
Registro com id = 1797182902765935718005078747253576 inserido com sucesso.
Registro com id = 1797182902765935718326579032099902 inserido com sucesso.
Registro com id = 1790650325691406718006275992961972 inserido com sucesso.
Registro com id = 1790650325691406718032597344644097 inserido com sucesso.
Registro com id = 1790650325691406717980478672546161 inserido com sucesso.
Registro com id = 1790650325691406717999631694815784 inserido com sucesso.
Registro com id = 1790650325691406717868068480908352 inserido com sucesso.
Registro com id = 1803239509675886518004555213694855 inserido com sucesso.
Registro com id = 1803239

C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table,

Registro com id = 1784327787615492417935132118501780 inserido com sucesso.
Registro com id = 1784327787615492417975602669999778 inserido com sucesso.
Registro com id = 1784327787615492417980419661785716 inserido com sucesso.
Registro com id = 1784327787615492417985627235816479 inserido com sucesso.
Registro com id = 1784327787615492417936678435580317 inserido com sucesso.
Registro com id = 1784327787615492417979201958875857 inserido com sucesso.
Registro com id = 1824220173121996117978190145738391 inserido com sucesso.
Registro com id = 1824220173121996118239417728149649 inserido com sucesso.
Registro com id = 1824220173121996117970852493869455 inserido com sucesso.
Registro com id = 1824220173121996117858477921823537 inserido com sucesso.
Registro com id = 1824220173121996117990323756717997 inserido com sucesso.
Registro com id = 1798855041555343717946312491417976 inserido com sucesso.
Registro com id = 1798855041555343717998050277570171 inserido com sucesso.
Registro com id = 1798855

C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)


Registro com id = 1793092968880379917993063657129058 inserido com sucesso.
Registro com id = 1793092968880379918242719579162133 inserido com sucesso.
Registro com id = 1793092968880379917960339570278289 inserido com sucesso.
Registro com id = 1793092968880379918196692265234329 inserido com sucesso.
Registro com id = 1793092968880379917932358189417551 inserido com sucesso.
Registro com id = 1793092968880379917933182687919850 inserido com sucesso.
Registro com id = 1793092968880379917865567998619722 inserido com sucesso.
Registro com id = 1793092968880379917847530503583685 inserido com sucesso.
Registro com id = 1790864587789859518043780708198048 inserido com sucesso.


C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table, con=connection, if_exists='append', index=False)
C:\Users\gabri\AppData\Local\Temp\ipykernel_2204\128530510.py:15: UserWarning: The provided table name 'DTbcarrossel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  row.to_frame().T.to_sql(table,

In [ ]:
# send_to_db(engine, table, pk_column, dataset)